In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP/Utils'))

import NLPUtils as nlp
from NLPUtils.datasets import imdb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [3]:
from NLPUtils.datasets.utils import NgramTextVectorizer

train_corpus = imdb.train_reader()
indices = nlp.datasets.utils.split_dev_kfolds(len(train_corpus),dev_size=.2,random_state=1273)
train_idx, dev_idx = indices[0]
dev_corpus, train_corpus = train_corpus.iloc[dev_idx[:2],:], train_corpus.iloc[train_idx[:3],:]

for text in train_corpus['comment']:
    print(text)
    print()

I tried to watch this movie in a military camp during an overseas mission, and let me tell you, you'll watch anything under those circumstances. Not this piece of sh*t though.<br /><br />The first five minutes set the tone by weak porn-movie quality acting, weird out-of-the-blue plot twists and unbelievable situations and behavior. It gets worse after that. This movie does not have one single saving grace, and yet it is not bad in a way that would make it funny to watch. It's just horrible. I've seen quite many movies in my life and I'm not one of those snobby know-all critics, I mean I'll enjoy most movies to some extent even if they're bad. This one... man.<br /><br />Steer _well_ clear of this one, my friend.
I saw this film opening weekend in Australia, anticipating with an excellent cast of Ledger, Edgerton, Bloom, Watts and Rush that the definitive story of Ned Kelly would unfold before me. Unfortunately, despite an outstanding performance by Heath Ledger in the lead role, the pl

In [7]:
import time

vocabulary = None#imdb.vocab_file_reader()
unk_token = 'UNK'
ngram_range = (1,1)
token_pattern = r'\b\w+\b'
min_freq, max_freq = 0, np.inf

vectorizer = NgramTextVectorizer(vocabulary=vocabulary,ngram_range=ngram_range,
                min_freq=min_freq,max_freq=max_freq,unk_token=unk_token)

vectorizer.fit(train_corpus['comment'])

X_train = vectorizer.transform(train_corpus['comment'])
X_dev = vectorizer.transform(dev_corpus['comment'])

4.76837158203125e-06
4.0531158447265625e-05
0.0035991668701171875
0.0012743473052978516
0.0009412765502929688


Tengo dos casos posibles:
* Tengo un vocabulario abierto, en el que lo único que hago es contar las palabras que aparecieron y me quedo con todas ellas. 

* Tengo un vocabulario cerrado. Esto se puede dar por varias razones:

    * Le doy un vocabulario explícito
    * Le doy una frecuencia máxima y/o mínima de ocurrencia
    * Le doy un máximo de palabras que pueden aparecer en el vocabulario (las más frecuentes).
    
    En todos estos casos, me quedo con las palabras que integran el vocabulario y el resto, las ignoro o las trato como UNKs, dependiendo del valor de unk_token

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_train = ['Esto es una prueba',
               'esto también',
               'esto es otra prueba']

corpus_dev = ['esto es una prueba para dev',
             'esto también.',
             'esto no importa pero igual']

vocabulary = ['esto','igual','es','una']

vec = CountVectorizer(vocabulary=vocabulary,token_pattern=r'\b\w+\b',lowercase=False)
vec.fit(corpus_train)
print(vec.vocabulary_)
X = vec.transform(corpus_dev)
X.toarray()

{'esto': 0, 'igual': 1, 'es': 2, 'una': 3}


array([[1, 0, 1, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0]])

In [86]:
from NLPUtils.datasets.utils import NgramTextVectorizer

train_corpus = ['Esto es una prueba',
               'esto también',
               'esto es otra prueba']

dev_corpus = ['esto es una prueba para dev',
             'esto también.',
             'esto no importa pero igual']


# train_corpus = imdb.train_reader()
# indices = nlp.datasets.utils.split_dev_kfolds(len(train_corpus),dev_size=.2,random_state=1273)
# train_idx, dev_idx = indices[0]
# dev_corpus, train_corpus = train_corpus.iloc[dev_idx,:], train_corpus.iloc[train_idx,:]

vocabulary = None#imdb.vocab_file_reader()[:10] #['esto', 'es', 'una', 'para', 'igual']
unk_token = 'UNK'
ngram_range = (1,3)
token_pattern = r'\b\w+\b'
min_freq, max_freq = 0, 1
max_features = 10
vectorizer = NgramTextVectorizer(vocabulary=vocabulary,ngram_range=ngram_range,
                min_freq=min_freq,max_freq=max_freq,unk_token=unk_token,max_features=max_features)
X_train = vectorizer.fit_transform(train_corpus)
X_dev = vectorizer.transform(dev_corpus)

print(vectorizer.vocabulary_)
print(X_train.shape)
print(X_dev.shape)
print(X_train.toarray())
print(X_dev.toarray())

{'una prueba': 0, 'una': 1, 'también': 2, 'otra prueba': 3, 'otra': 4, 'esto también': 5, 'esto es otra': 6, 'esto es': 7, 'es una prueba': 8, 'es una': 9, 'UNK': 10}
(3, 11)
(3, 11)
[[1 1 0 0 0 0 0 0 1 1 5]
 [0 0 1 0 0 1 0 0 0 0 1]
 [0 0 0 1 1 0 1 1 0 0 5]]
[[ 1  1  0  0  0  0  0  1  1  1 10]
 [ 0  0  1  0  0  1  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0 12]]


In [61]:
train_corpus

,comment,rate
5192,I tried to watch this movie in a military camp...,1
10582,"I saw this film opening weekend in Australia, ...",4
14511,St. Elmo's Fire has no bearing on life after u...,1
21452,Bette Davis brings her full trunk of tics to t...,1
11081,I'd heard about this movie a while ago from a ...,3
17787,"I really can't remember who recommended this, ...",8
14935,"According to IMDb, as well as to every other w...",4
23827,This movie does a great job of explaining the ...,10
24267,When I first saw the Romeo Division last sprin...,10
10051,This movie could be used in film classes in a ...,2
